In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

In [3]:
import metpy.calc as mpcalc
from metpy.units import units
import metpy

def calculate_radiative_heating(swnet_top, swdn_sfc, lwnet_top, lwdn_sfc, 
                              pressure, temperature):
    """
    Calculate radiative heating rate profile.

    Parameters:
    -----------
    swnet_top : array-like, shape (time, )
        Net shortwave radiation at the top of the atmosphere
    swdn_sfc : array-like, shape (time, )
        Downward shortwave radiation at the surface
    lwnet_top : array-like, shape (time, )
        Net longwave radiation at the top of the atmosphere
    lwdn_sfc : array-like, shape (time, )
        Downward longwave radiation at the surface
    pressure : array-like, shape (levels, )
        Pressure levels in Pa
    temperature : array-like, shape (time, levels)
        Temperature at the center point in K
    """
    # Calculate the radiative heating rate
    dT_dt_rad = np.zeros((swnet_top.shape[0], pressure.shape[0]))
    for t in range(swnet_top.shape[0]):
        # Calculate the radiative heating rate
        dT_dt_rad[t, :] = (swnet_top[t] - swdn_sfc[t] + lwnet_top[t] - lwdn_sfc[t]) / (pressure * temperature[t, :])
    return dT_dt_rad


def calculate_grid_spacing(lats, lons, center_idx=1):
    """
    Calculate grid spacing in meters, accounting for latitude.
    
    Parameters:
    -----------
    lats : array-like, shape (3,)
        Latitude values for 3 grid points
    lons : array-like, shape (3,)
        Longitude values for 3 grid points
    center_idx : int
        Index of the center point (default=1)
        
    Returns:
    --------
    dx : float
        Grid spacing in x-direction (meters) at center latitude
    dy : float
        Grid spacing in y-direction (meters)
    """
    # Earth's radius in meters
    R_EARTH = 6371000
    
    # Center latitude for dx calculation
    center_lat = lats[center_idx]
    # print(f"lats: {lats}, lons: {lons}, center_idx: {center_idx}")
    # Convert to radians
    lat_rad = np.deg2rad(center_lat)
    
    # Calculate dy (latitude spacing)
    # For 1-degree latitude difference
    dy = R_EARTH * np.deg2rad(np.abs(lats[2] - lats[0])) / 2
    
    # Calculate dx (longitude spacing)
    # dx varies with latitude due to convergence of meridians
    # For 1-degree longitude difference at the center latitude
    dx = R_EARTH * np.cos(lat_rad) * np.deg2rad(np.abs(lons[2] - lons[0])) / 2

    return dx, dy


def calculate_horizontal_gradients(var, lats, lons, center_idx=1):
    """
    Calculate horizontal gradients using centered differences accounting for latitude.
    
    Parameters:
    -----------
    var : array-like, shape (time, level, 3, 3)
        Variable on 3x3 grid centered on column of interest
    lats : array-like, shape (3,)
        Latitude values for 3 grid points
    lons : array-like, shape (3,)
        Longitude values for 3 grid points
    center_idx : int
        Index of the center point (default=1)
        
    Returns:
    --------
    d_dx : array-like, shape (time, level)
        Zonal gradient at center point
    d_dy : array-like, shape (time, level)
        Meridional gradient at center point
    """
    # Get proper grid spacing at center latitude
    dx, dy = calculate_grid_spacing(lats, lons, center_idx)
    # print(f"dx: {dx}, dy: {dy}") 
    # Calculate centered differences at the middle point
    d_dx = (var.isel(latitude=1, longitude=2, drop=True) - var.isel(latitude=1, longitude=0, drop=True)) / (2 * dx)
    d_dy = (var.isel(latitude=2, longitude=1, drop=True) - var.isel(latitude=0, longitude=1, drop=True)) / (2 * dy)
    # print(f"d_dx: {d_dx}, d_dy: {d_dy}")
    return d_dx, d_dy


def calculate_geostrophic_wind(z, lat, lon):
    """
    Calculate geostrophic wind components from geopotential height.
    
    Parameters:
    -----------
    z : array-like, shape (time, level, 3, 3)
        Geopotential height on 3x3 grid
    lat : array-like, shape (3, 3)
        Latitudes of grid points
    lon : array-like, shape (3, 3)
        Longitudes of grid points
    
    Returns:
    --------
    u_g : array-like, shape (time, level)
        Zonal geostrophic wind at center point
    v_g : array-like, shape (time, level)
        Meridional geostrophic wind at center point
    """
    # Coriolis parameter at center point
    f = 2 * 7.2921e-5 * np.sin(np.deg2rad(lat[1]))
    
    # Calculate height gradients with proper spacing
    dz_dx, dz_dy = calculate_horizontal_gradients(z, lat, lon)
    
    # Calculate geostrophic wind components
    u_g = -(1/f) * 9.81 * dz_dy
    v_g = (1/f) * 9.81 * dz_dx
    
    return u_g, v_g


def theta_from_t(temperature, pressure):
    """Convert temperature to potential temperature."""
    return mpcalc.potential_temperature(
        pressure * units.Pa, 
        temperature * units.kelvin
    ).metpy.dequantify()


def rho_from_sp(sp, t):
    """Calculate air density from surface pressure and temperature."""
    return sp / (287.05 * t)


def calculate_advection(var, u, v, w, lats, lons, pressure, temperature, center_idx=1):
    """
    Calculate advection terms using 3x3 grid with proper grid spacing.
    
    Parameters:
    -----------
    var : array-like, shape (time, level, 3, 3)
        Variable to calculate advection for
    u : array-like, shape (time, level)
        Zonal wind at center point
    v : array-like, shape (time, level)
        Meridional wind at center point
    w : array-like, shape (time, level)
        Vertical velocity at center point
    lats : array-like, shape (3,)
        Latitude values for 3 grid points
    lons : array-like, shape (3,)
        Longitude values for 3 grid points
    pressure : array-like, shape (level,)
        Pressure levels in Pa
    temperature : array-like, shape (time, level)
        Temperature at center point in K
    center_idx : int
        Index of the center point (default=1)
    """
    # Calculate horizontal gradients with proper spacing
    # print(f"lats: {lats}, lons: {lons}, center_idx: {center_idx}")
    dvardx, dvardy = calculate_horizontal_gradients(var, lats, lons, center_idx)
    
    # Calculate heights for vertical gradient
    pressure_pa = pressure * units.Pa
    temperature = temperature * units.kelvin
    
    heights = np.zeros((var.shape[0], var.shape[1]))
    for t in range(var.shape[0]):
        # convert km to m
        # print(mpcalc.pressure_to_height_std(pressure_hpa).magnitude)
        heights[t, :] = mpcalc.pressure_to_height_std(pressure_pa).magnitude * 1000
    
    # Calculate vertical gradient (at center point)
    dvardz = xr.DataArray(np.zeros((var.shape[0], var.shape[1])), dims=('time', 'levels'))
    for t in range(var.shape[0]):
        dvardz[t, :] = np.gradient(var[t, :, center_idx, center_idx], heights[t, :])
        if t ==0:
            print(f"heights for time {t} is {heights[t,:]}")
            print(f"dvardz is {dvardz[t, :]}")
            print(f"v_advec is {-w * dvardz}")
    
    # Calculate advection terms
    h_advec = -(u * dvardx + v * dvardy)
    v_advec = -w * dvardz
    
    return h_advec, v_advec

In [4]:
import metpy.constants


def calculate_thetal(theta, t, q):
    """
    Calculate liquid water equivalent potential temperature (thetal) from potential temperature, temperature, and specific humidity.
    Parameters:
    -----------
    theta : array-like, shape (time, level, latitude, longitude)
        Potential temperature in K
    t : array-like, shape (time, level, latitude, longitude)
        Temperature in K
    q : array-like, shape (time, level, latitude, longitude)
        Specific humidity in kg/kg
    Returns:
    --------
    thetal : array-like, shape (time, level, latitude, longitude)
        Liquid water equivalent potential temperature in K
    """
    # cpd is specific heat of dry air at constant pressure
    t = t * units.kelvin  # Ensure temperature is in Kelvin
    cpd = metpy.constants.Cp_d
    # Lu is the latent heat of vaporization
    Lu = mpcalc.water_latent_heat_vaporization(t)
    # rl is the mixing ratio
    rl = mpcalc.mixing_ratio_from_specific_humidity(q * units.kg / units.kg)
    # Calculate the liquid water equivalent potential temperature
    thetal = theta - (theta/ t) * (Lu * rl / cpd)
    print(f"theta is {theta}")
    print(f"t is {t}")
    print(f"Lu is {Lu}")
    print(f"rl is {rl}")
    print(f"cpd is {cpd}")
    return thetal.metpy.dequantify()  # Convert to numpy array in Kelvin

    
    


In [5]:
def expand_pressure_3d(pressure_level, t):
    """
    Expand pressure levels to 3D array matching the time dimension.
    
    Parameters:
    -----------
    pressure_level : array-like, shape (levels, )
        Pressure levels in Pa
    t : array-like, shape (time, levels, latitude, longitude)
        4D array with time, levels, latitude, and longitude dimensions.
    
    Returns:
    --------
    expanded_pressure : array-like, shape (time, levels)
        Expanded pressure levels
    """
    temp1 = np.tile(pressure_level, (t.shape[0], 1))
    temp2 = np.tile(temp1, (t.shape[2], t.shape[3], 1, 1))

    da_pressure = xr.DataArray(
        temp2,
        dims=('latitude', 'longitude', 'time', 'levels'),
        coords={
            'time': t.time,
            'levels': t.levels,
            'latitude': t.latitude,
            'longitude': t.longitude
        })

    da_pressure = da_pressure.transpose('time', 'levels', 'latitude', 'longitude')
    return da_pressure

In [7]:
def era5_to_scm_driver(ds):
    # Create output dataset
    out = xr.Dataset()
    pressure_levels = ds.levels * 100  # convert hPa to Pa
    out.coords['levels'] = pressure_levels
    ds = ds.update({'levels':pressure_levels})
    pressure_levels = ds.levels
    # print(ds.levels)

    # read in soil moisture and temperature
    soil_moist = []
    soil_temp = []
    for i in range(1,5):
        da = ds[f"swvl{i}"]
        da = da.expand_dims(dim={"nsoil": 1})
        soil_moist.append(da)
        da_t = ds[f"stl{i}"]
        da_t = da_t.expand_dims(dim={"nsoil": 1})
        soil_temp.append(da_t)
    da_soil_moist_sum = xr.concat(soil_moist,dim='nsoil')
    da_soil_temp_sum = xr.concat(soil_temp,dim='nsoil')
    
    # era5_noahmp_exp_SCM_driver.test11.nc
    geopotential_height_m = mpcalc.geopotential_to_height(ds.z * units(ds.z.attrs['units'])).metpy.dequantify()
    
    # convert the vertical velocity from Pa/s to m/s
    w = ds_in.w.isel(latitude=1,longitude=1,drop=True)
    t = ds_in.t.isel(latitude=1,longitude=1,drop=True)
    q = ds_in.q.isel(latitude=1,longitude=1,drop=True)
    omega = w * units.Pa / units.second
    pressure_2d = xr.DataArray(np.tile(pressure_levels, (len(w.time),1)),
                               dims = w.dims,
                               coords = w.coords) * units.Pa
    mixing_ratio = mpcalc.mixing_ratio_from_specific_humidity(q * units.kilogram / units.kilogram)
    w_m_s = mpcalc.vertical_velocity(omega, 
                                     pressure_2d, 
                                     t * units.kelvin, 
                                     mixing_ratio).metpy.dequantify()
    print(w_m_s)
    height_surface = mpcalc.pressure_to_height_std(ds.sp * units.Pa).metpy.dequantify() * 1000
    u_g_surf, v_g_surf = calculate_geostrophic_wind(height_surface, 
                                          ds.latitude, ds.longitude)
    u_g, v_g = calculate_geostrophic_wind(mpcalc.geopotential_to_height(ds.z).metpy.dequantify(), 
                                          ds.latitude, ds.longitude)
    # print(ds.t)
    # print(pressure_levels)
    # print(theta_from_t(ds.t, pressure_levels).transpose('time', 'levels', 'latitude', 'longitude'))
    theta = theta_from_t(ds.t, pressure_levels).transpose('time', 'levels', 'latitude', 'longitude')
    print(f"ds.t is {ds.t}")
    print(f"pressure level is {pressure_levels}")
    print(f"calculated theta is {theta.isel(latitude=1,longitude=1).sel(levels=85000)}")
    h_advec_theta, v_advec_theta = calculate_advection(
        theta,
        ds.u.values[..., 1, 1],  # center point
        ds.v.values[..., 1, 1],
        w_m_s,
        ds['latitude'].values,
        ds['longitude'].values,
        ds.levels.values,
        ds.t.values[..., 1, 1]
    )

    h_advec_qt, v_advec_qt = calculate_advection(
        ds.q,
        ds.u.isel(latitude=1, longitude=1).values,
        ds.v.isel(latitude=1, longitude=1).values,
        w_m_s,
        ds['latitude'].values,
        ds['longitude'].values,
        ds.levels.values,
        ds.t.isel(latitude=1, longitude=1).values
    )

    # Convert omega to w
    rho = ds.sp / (287.0 * ds.t)  # approximate air density
    w = -ds.w / (rho * 9.81)
    out['w_ls'] = w_m_s
    out['omega'] = ds.w.isel(latitude=1, longitude=1)
    out['geopotential_height'] = geopotential_height_m.isel(latitude=1, longitude=1)

    # Nudging fields
    out['u_nudge'] = ds.u.isel(latitude=1, longitude=1)
    out['v_nudge'] = ds.v.isel(latitude=1, longitude=1)
    out['T_nudge'] = ds.t.isel(latitude=1, longitude=1)
    out['Sensible_Heat_nudge'] = ds.sshf.isel(latitude=1, longitude=1)
    out['thil_nudge'] = (('time', 'levels'), 
                         theta_from_t(ds.t.isel(latitude=1, longitude=1), pressure_levels).values.T)
    out['qt_nudge'] = ds.q.isel(latitude=1, longitude=1)

    out['u_g'] = u_g
    out['v_g'] = v_g
    out['u_g_surf'] = u_g_surf
    out['v_g_surf'] = v_g_surf
    out['h_advec_thetail'] = h_advec_theta
    out['v_advec_thetail'] = v_advec_theta
    # fix this error: era5_noahmp_exp_SCM_driver.test6.nc
    out['h_advec_qt'] = h_advec_qt
    out['v_advec_qt'] = v_advec_qt
    out['p_surf'] = ds.sp.isel(latitude=1, longitude=1, drop=True)
    out['T_surf'] = ds.t2m.isel(latitude=1, longitude=1, drop=True)

    # Calculate radiative heating rate
    swnet_top = ds.tsr.isel(latitude=1, longitude=1).values
    # Possibly below should be `ds.ssrd`
    swdn_sfc = ds.ssr.isel(latitude=1, longitude=1).values
    lwnet_top = ds.ttr.isel(latitude=1, longitude=1).values
    # Possibly below should be `ds.strd`
    lwdn_sfc = ds.str.isel(latitude=1, longitude=1).values
    
    # Reshape pressure levels to match the time dimension
    pressure_2d = np.tile(pressure_levels.values, (swnet_top.shape[0], 1))
    # print(pressure_2d.shape)
    temperature = ds.t.isel(latitude=1, longitude=1).values
    
    # Calculate radiative heating rate
    dT_dt_rad = calculate_radiative_heating(
        swnet_top, swdn_sfc, lwnet_top, lwdn_sfc, 
        pressure_levels, temperature
    )
    # print("dT_dt_rad shape:", dT_dt_rad.shape)
    
    # Add to output dataset
    out['dT_dt_rad'] = (('time', 'levels'), dT_dt_rad)
    
    # Calculate Air Liquid Potential Temperature
    # Potential temperature: out['thil_nudge']
    p_3d = expand_pressure_3d(pressure_levels, ds.t)
    # print(pressure_levels)
    # print(p_3d)
    # test9: era5_noahmp_exp_SCM_driver.test9.nc
    thetal = theta_from_t(ds.t, p_3d) #calculate_thetal(theta_from_t(ds.t, p_3d), ds.t, ds.q)
    # print(thetal)

    h_advec_thetal, v_advec_thetal = calculate_advection(
        thetal,
        ds.u.values[..., 1, 1],  # center point
        ds.v.values[..., 1, 1],
        w_m_s,
        ds['latitude'].values,
        ds['longitude'].values,
        ds.levels.values,
        ds.t.values[..., 1, 1]
    )

    h_advec_p, v_advec_p = calculate_advection(
        p_3d,
        ds.u.values[..., 1, 1],  # center point
        ds.v.values[..., 1, 1],
        w_m_s,
        ds['latitude'].values,
        ds['longitude'].values,
        ds.levels.values,
        ds.t.values[..., 1, 1]
    )

    # wap: lagrangian_tendency_of_air_pressure

    out['thetal'] = thetal.isel(latitude=1, longitude=1, drop=True)
    out['h_advec_thetal'] = h_advec_thetal
    out['v_advec_thetal'] = v_advec_thetal
    out['h_advec_p'] = h_advec_p
    out['v_advec_p'] = v_advec_p
    out['p'] = p_3d.isel(latitude=1, longitude=1)
    # https://codes.ecmwf.int/grib/param-db/135
    out['wap'] = ds.w.isel(latitude=1, longitude=1)
    
    out = out.transpose('levels', 'time')

    # add soil conditions
    out['soil_moisture'] = da_soil_moist_sum.isel(latitude = 1, longitude = 1, drop=True)
    out['soil_temperature'] = da_soil_temp_sum.isel(latitude = 1, longitude = 1, drop=True)

    # Add variable attributes
    var_attrs = {
        'p_surf': {'units': 'Pa', 'long_name': 'surface pressure'},
        'T_surf': {'units': 'K', 'long_name': 'surface absolute temperature'},
        'w_ls': {'units': 'm s^-1', 'long_name': 'large scale vertical velocity'},
        'geopotential_height': {'units':'m', 'long_name': 'geopotential height'},
        'omega': {'units': 'Pa s^-1', 'long_name': 'large scale pressure vertical velocity'},
        'u_g': {'units': 'm s^-1', 'long_name': 'large scale geostrophic E-W wind'},
        'v_g': {'units': 'm s^-1', 'long_name': 'large scale geostrophic N-S wind'},
        'u_g_surf': {'units': 'm s^-1', 'long_name': 'large scale geostrophic E-W wind at surface'},
        'v_g_surf': {'units': 'm s^-1', 'long_name': 'large scale geostrophic N-S wind at surface'},
        'u_nudge': {'units': 'm s^-1', 'long_name': 'E-W wind to nudge toward'},
        'v_nudge': {'units': 'm s^-1', 'long_name': 'N-S wind to nudge toward'},
        'T_nudge': {'units': 'K', 'long_name': 'absolute temperature to nudge toward'},
        'Sensible_Heat_nudge': {'units': 'W m^-2', 'long_name': 'sensible heat fluxes'},
        'thil_nudge': {'units': 'K', 'long_name': 'potential temperature to nudge toward'},
        'qt_nudge': {'units': 'kg kg^-1', 'long_name': 'q_t to nudge toward'},
        'dT_dt_rad': {'units': 'K s^-1', 'long_name': 'prescribed radiative heating rate'},
        'h_advec_thetail': {'units': 'K s^-1', 'long_name': 'prescribed theta_il tendency due to horizontal advection'},
        'v_advec_thetail': {'units': 'K s^-1', 'long_name': 'prescribed theta_il tendency due to vertical advection'},
        'h_advec_qt': {'units': 'kg kg^-1 s^-1', 'long_name': 'prescribed q_t tendency due to horizontal advection'},
        'v_advec_qt': {'units': 'kg kg^-1 s^-1', 'long_name': 'prescribed q_t tendency due to vertical advection'},
        'h_advec_thetal': {'units': 'K s^-1', 'long_name': 'prescribed theta_l tendency due to horizontal advection'},
        'v_advec_thetal': {'units': 'K s^-1', 'long_name': 'prescribed theta_l tendency due to vertical advection'},
        'h_advec_p': {'units': 'Pa s^-1', 'long_name': 'prescribed pressure tendency due to horizontal advection'},
        'v_advec_p': {'units': 'Pa s^-1', 'long_name': 'prescribed pressure tendency due to vertical advection'},
        'thetal': {'units': 'K', 'long_name': 'liquid water equivalent potential temperature'},
        'p': {'units': 'Pa', 'long_name': 'pressure at center point'},
        'wap': {'units': 'Pa s^-1', 'long_name': 'Lagrangian tendency of air pressure'},
        'soil_moisture': {'units': 'kg kg^-1', 'long_name': 'Volumetric soil water'},
        'soil_temperature': {'units': 'K', 'long_name': 'Soil temperature'}
    }
    
    for var in out.variables:
        if var in var_attrs:
            out[var].attrs = var_attrs[var]
 
    return out

In [9]:
infile = "/glade/derecho/scratch/yifanc/wpo2023/SCM/processed/US-Whs/processed_era5.2019.US-Whs.nc"
ds_in = xr.open_dataset(infile)

In [11]:
ds_in = ds_in.rename({'valid_time': 'time', 'pressure_level': 'levels'})
ds_in = ds_in.sel(time=slice("2019-02-01 00","2019-02-28 23"))

In [12]:
ds_out = era5_to_scm_driver(ds_in)

<xarray.DataArray (time: 672, levels: 37)> Size: 199kB
array([[ 0.01517533, -0.1015591 ,  0.00032224, ...,  0.00640667,
         0.00688262,  0.00693248],
       [ 0.03415979, -0.07851875, -0.09391407, ...,  0.00113029,
         0.0011807 ,  0.00135146],
       [ 0.06857207, -0.02113392, -0.04847617, ...,  0.00091982,
         0.00093471,  0.00090257],
       ...,
       [ 0.13737994, -0.03709961, -0.02684706, ..., -0.00137338,
        -0.0011355 , -0.00090948],
       [ 0.07428513, -0.01212298,  0.07172614, ..., -0.00306901,
        -0.00298206, -0.00263289],
       [ 0.05098056,  0.04590175,  0.01922165, ..., -0.00405726,
        -0.00398926, -0.00371641]], shape=(672, 37))
Coordinates:
  * levels   (levels) float64 296B 100.0 200.0 300.0 ... 9.5e+04 9.75e+04 1e+05
  * time     (time) datetime64[ns] 5kB 2019-02-01 ... 2019-02-28T23:00:00
Attributes:
    units:    meter / second
ds.t is <xarray.DataArray 't' (time: 672, levels: 37, latitude: 3, longitude: 3)> Size: 895kB
array([[[[257

In [13]:
ds_out.to_netcdf("/glade/derecho/scratch/yifanc/wpo2023/SCM/processed/US-Whs/processed_era5.201902.US-Whs.derecho.nc")